In [12]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
imagelist=['amber1.png','amber2.png','amy1.png','amy2.png','andrew1.png','andrew2.png','andy1.png','andy2.png','erin1.png','erin2.png','gabe1.png','gabe2.png','hill2.png','hill4.png','jack1.png','jack2.png','zach1.png','zach2.png']

train_img=[]
for file in imagelist:
    first=np.array(imageio.imread('Train/'+file))
    m,n=first.shape
    first=np.reshape(first,(m*n,1))
    train_img.append(first)

print(m,n)
img_train=np.matrix(np.array(train_img))
train_img=np.transpose(img_train)
print(train_img)
m,n=train_img.shape
print("train_img.shape=",m,n)

##calculating average face vector
avgface=np.ones((m,1))
for i in range(0,m):
    avgface[i]=np.mean(train_img[i])
print("The average face vector is",avgface)
storedavgface=avgface

##subtracting average face vector from all faces
for i in range(0,n):
    norm=np.transpose(train_img) - np.transpose(avgface)

150 125
[[ 28  22  13 ... 220 163 197]
 [ 38  20  20 ... 217 160 188]
 [ 38  26  27 ... 224 149 193]
 ...
 [ 14  15 174 ... 197 106  69]
 [  9  22 126 ... 237 123  72]
 [ 20  20 128 ... 252 123  75]]
train_img.shape= 18750 18
The average face vector is [[124.16666667]
 [129.11111111]
 [135.44444444]
 ...
 [ 81.66666667]
 [ 85.44444444]
 [ 90.05555556]]


In [29]:
##calculating covariance matrix
covmat=(norm*np.transpose(norm))
print("The covariance matrix is")
#print(covmat)

##calculating eigen values out of covariance matrix
eigen,u=np.linalg.eig(covmat)
print("U matrix is")
#print(v)
print("eigen values are")
print(eigen)

a,s,v=np.linalg.svd(covmat)

##determining value of k i.e.(no. of principal components)
summation=np.sum(s)
print(summation)
for k in range(1,len(s)):
    sigma=0
    for i in range(0,k):
        sigma+=s[i]
    sigma=sigma/summation
    if(sigma>=0.95):
        print("selected value of k is",k)
        break

##selecting k eigen vectors from the set of 18 vectors
eigenvectors=np.empty((k,18))
for i in range(0,k):
    temp=np.empty((1,18))
    for j in range(0,18):
        temp[0,j]=u[i,j]
    eigenvectors[i]=temp

#print("selected eigen vectors are",eigenvectors)

##mapping selected eigen vectors to original image size in training set
mappedeigenvectors=train_img*np.transpose(eigenvectors)
print(mappedeigenvectors.shape)

weightmatrix=np.transpose(mappedeigenvectors)*np.transpose(norm)
print(weightmatrix.shape)

The covariance matrix is
U matrix is
eigen values are
[ 2.88200692e+08  1.42556299e+08  1.15553499e+08  8.05023466e+07
  6.50087905e+07  4.13938151e+07  3.63234081e+07  3.29389871e+07
  2.20785271e+07  1.67189305e+07 -2.77224923e-09  1.37716275e+07
  3.28206084e+06  3.89992952e+06  5.38354016e+06  8.92780384e+06
  8.53907155e+06  7.39918437e+06]
892478512.2777779
selected value of k is 11
(18750, 11)
(11, 18)


In [19]:
##calculating overall average face vector
transposeweightmatrix=np.transpose(weightmatrix)
avgface=np.ones((11,1))
for i in range(0,11):
    avgface[i]=np.mean(transposeweightmatrix[i])
print("The average face vector is",avgface)

##calculating average face vector for each class
classavgface=np.ones((11,9))
for j in range(0,9):
    for i in range(0,11):
        classavgface[i][j]=(int)((weightmatrix[i,2*j]/2+weightmatrix[i,2*j+1]/2))
transclassavg=np.transpose(classavgface)
#print("sdjfkl",transclassavg.shape)
storedtransclassavg=transclassavg

##subtracting average class face vector from each of the class vector
j=0
for i in range(0,n):
    transposeweightmatrix[i]=transposeweightmatrix[i]-transclassavg[j]
    if(i%2==0 and i!=0):
        j+=1


##subtracting total average face vector from class average face vector
for i in range(0,9):
    transclassavg[i]=transclassavg[i]-np.transpose(avgface)

##calculating between class scatter matrix
print(transclassavg[4].shape)
sbetweenclass=np.zeros((11,11))
for i in range(0,9):
    sbetweenclass+=np.transpose(transclassavg[i])*transclassavg[i]


##calculating within class scatter matrix
s1=np.transpose(transposeweightmatrix[0])*transposeweightmatrix[0]+np.transpose(transposeweightmatrix[1])*transposeweightmatrix[1]
s2=np.transpose(transposeweightmatrix[2])*transposeweightmatrix[2]+np.transpose(transposeweightmatrix[3])*transposeweightmatrix[3]
s3=np.transpose(transposeweightmatrix[4])*transposeweightmatrix[4]+np.transpose(transposeweightmatrix[5])*transposeweightmatrix[5]
s4=np.transpose(transposeweightmatrix[6])*transposeweightmatrix[6]+np.transpose(transposeweightmatrix[7])*transposeweightmatrix[7]
s5=np.transpose(transposeweightmatrix[8])*transposeweightmatrix[8]+np.transpose(transposeweightmatrix[9])*transposeweightmatrix[9]
s6=np.transpose(transposeweightmatrix[10])*transposeweightmatrix[10]+np.transpose(transposeweightmatrix[11])*transposeweightmatrix[11]
s7=np.transpose(transposeweightmatrix[12])*transposeweightmatrix[12]+np.transpose(transposeweightmatrix[13])*transposeweightmatrix[13]
s8=np.transpose(transposeweightmatrix[14])*transposeweightmatrix[14]+np.transpose(transposeweightmatrix[15])*transposeweightmatrix[15]
s9=np.transpose(transposeweightmatrix[16])*transposeweightmatrix[16]+np.transpose(transposeweightmatrix[17])*transposeweightmatrix[17]
swithinclass=s1+s2+s3+s4+s5+s6+s7+s8+s9
print(swithinclass.shape)
print(sbetweenclass.shape)

The average face vector is [[  8462726.4971443 ]
 [ -8462726.37031683]
 [ -4193708.16373361]
 [   197456.39797178]
 [   669960.08978024]
 [   119117.61120547]
 [-12168053.94024777]
 [   563030.80642228]
 [  4776324.77429984]
 [  -798039.40040284]
 [-18671115.73012321]]
(11,)
(11, 11)
(11, 11)


In [27]:
##calculating eigenvectors of given images
temp=np.linalg.inv(swithinclass)*sbetweenclass
ldaeigenvectors,D=np.linalg.eig(temp)
print("ldaeigenvectors.shape=",ldaeigenvectors.shape)

##projecting faces onto lda-space
projectedldaeigenvectors=ldaeigenvectors*weightmatrix
print("projectedldaeigenvectors.shape=",projectedldaeigenvectors.shape)

###Testing unknown face
image=imageio.imread('Test/zach3.png')
m,n=image.shape
image=np.reshape(image,(m*n,1))
normalisedface=image-storedavgface
print(normalisedface.shape)


testweight=np.transpose(mappedeigenvectors)*normalisedface
print(testweight.shape)
ldatestweight=np.transpose(projectedldaeigenvectors)*np.transpose(testweight)

transposeweightmatrix=np.transpose(weightmatrix)
m,n=transposeweightmatrix.shape


flag=0
store=[]
for i in range(0,m):
    difference=transposeweightmatrix[i]-np.transpose(testweight)
    totaldiff=np.sum(np.power(difference,2))
    totaldiff=np.sqrt(totaldiff)
    print(totaldiff)
    store.append(totaldiff)
    if(totaldiff <= min(store)):
        pos = i
    
a=min(store)

print("Minimum Distance = ",a)


ldaeigenvectors.shape= (11,)
projectedldaeigenvectors.shape= (1, 18)
(18750, 1)
(11, 1)
129849792.33574753
60062526.84524301
171492497.83278802
82394643.19698642
117037700.82511432
96012557.35279109
144332425.90939587
149549472.51531032
200780441.4461046
62790759.57874011
80217750.17316496
123634871.63960116
154930064.73612332
89967338.10218018
132659821.71231504
88180913.04896842
82010891.43793447
131187831.76224408
Minimum Distance =  60062526.84524301
